In [4]:
# 1) Imports
import numpy as np
import pandas as pd
import pickle


In [5]:
# 2) Carregar artefatos treinados
MODEL_PATH = "artifacts/model.pkl"
THRESH_PATH = "artifacts/threshold.pkl"

with open(MODEL_PATH, "rb") as f:
    MODEL = pickle.load(f)
with open(THRESH_PATH, "rb") as f:
    THRESH = float(pickle.load(f))


prep = MODEL.named_steps["prep"]
NUM_COLS = list(prep.transformers_[0][2])  
CAT_COLS = list(prep.transformers_[1][2])  


In [6]:
# 3) Função de predição — retorna SOMENTE {"default": 0 ou 1}
def predict_default(input_dict: dict) -> dict:
    X = pd.DataFrame([input_dict])

    
    for c in NUM_COLS + CAT_COLS:
        if c not in X.columns:
            X[c] = np.nan
    X = X[NUM_COLS + CAT_COLS]


    for c in NUM_COLS:
        X[c] = pd.to_numeric(X[c], errors="coerce")
    for c in CAT_COLS:
        X[c] = X[c].astype("object")


    p = float(MODEL.predict_proba(X)[:, 1])
    return {"default": int(p >= THRESH)}


In [7]:
# 4) Exemplo de uso (remova na entrega se quiser)
predict_default({"ioi_3months": 3, "valor_vencido": 152000, "valor_total_pedido": 35000})


{'default': 1}